In [9]:
import requests
from bs4 import BeautifulSoup

In [10]:
url1 = "https://www.topuniversities.com/university-rankings/world-university-rankings/2018"
url2 = "https://www.timeshighereducation.com/world-university-rankings/2018/world-ranking#!/page/0/length/100/sort_by/rank/sort_order/asc/cols/stats"
json1 = "https://www.topuniversities.com//sites//default//files//qs-rankings-data//357051.txt"
json2 = "https://www.timeshighereducation.com//sites//default//files//the_data_rankings//world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json"
r = requests.get(json1)
json1_text = r.json()
r = requests.get(json2)
json2_text = r.json()

In [11]:
import pandas as pd
import numpy as np
num_of_rank = 200
data1 = pd.DataFrame(data=json1_text["data"][0:200], index=list(range(num_of_rank)))
data2 = pd.DataFrame(data=json2_text["data"][0:200], index=list(range(num_of_rank)))

In [12]:
d1 = data1.copy(deep=True)
d2 = data2.copy(deep=True)
d1.drop(['cc', 'core_id', 'guide', 'logo', 'nid', 'stars'], axis=1, inplace=True)
d2.drop(['aliases', 'member_level', 'nid', 'rank_order', 'record_type', 'scores_citations_rank', 'scores_citations_rank', 'scores_industry_income_rank',
       'scores_international_outlook_rank', 'scores_overall_rank', 'scores_research_rank', 'scores_teaching_rank', 'subjects_offered', 'url'], axis=1, inplace=True)

In [13]:

url_base = "https://www.topuniversities.com"
div1 = '<div class="faculty-main wrapper col-md-4">'
div2 = '<div class="students-main wrapper col-md-4">'
div3 = '<div class="int-students-main wrapper col-md-4">'
class_name = ["faculty-main wrapper col-md-4", "students-main wrapper col-md-4", "int-students-main wrapper col-md-4"]

In [14]:
d1_ = d1.copy(deep=True)
d1_ = d1_.assign(Total_Faculty=0)
d1_ = d1_.assign(International_Faculty=0)
d1_ = d1_.assign(Total_Student=0)
d1_ = d1_.assign(International_Student=0)

for i in d1_.index:
    url_ = d1_.iloc[i]['url']
    r = requests.get(url_base + url_)
    soup = BeautifulSoup(r.text, 'html.parser')
    lst = []
    for ic, c in enumerate(class_name):
        a = soup.findAll("div", { "class" : c})
        if len(a) > 0:
            for j in a[0].findAll("div", { "class" : "number" }):
                lst.append(int(''.join(ch for ch in j.get_text() if ch.isdigit())))
        if ic == 0:
            lst += [np.nan] * (2-len(lst))
        if ic == 1:
            lst += [np.nan] * (3-len(lst))
        if ic == 2:
            lst += [np.nan] * (4-len(lst))                
    d1_.loc[i, ['Total_Faculty', 'International_Faculty', 'Total_Student', 'International_Student']] = lst


In [24]:
QS = d1_.copy()
Times = d2.copy()

In [54]:
banned = set(['University', 'of', 'The'])
QS_words = QS['title'].apply(lambda x: x.split(' ')).apply(lambda x: list(set(x) - banned)).tolist()Times_words = Times['name'].apply(lambda x: x.split(' ')).apply(lambda x: list(set(x) - banned)).tolist()

In [70]:
points = [[len(set(qs_word_list) & set(times_word_list)) for qs_word_list in QS_words] for times_word_list in Times_words]
len(points)

200

In [78]:
qs_ids = [i for i in range(200)]
times_ids = [i for i in range(200,400)]

used_names = []

points = [[len(set(qs_word_list) & set(times_word_list)) for qs_word_list in QS_words] for times_word_list in Times_words]
for idx, point in enumerate(points):
    where = point.index(max(point))
    
    if where not in used_names:
        used_names.append(where)
        times_ids[idx] = where if max(point) > 0 else times_ids[idx]
        
QS['nameIndex'] = qs_ids
Times['nameIndex'] = times_ids


In [86]:
result = pd.merge(QS, Times, how='inner', on=['nameIndex'])
result[['name','title']]

,name,title
0,Massachusetts Institute of Technology,Massachusetts Institute of Technology (MIT)
1,Stanford University,Stanford University
2,Harvard University,Harvard University
3,California Institute of Technology,California Institute of Technology (Caltech)
4,University of Cambridge,University of Cambridge
5,University of Oxford,University of Oxford
6,Imperial College London,Imperial College London
7,University of Chicago,University of Chicago
8,ETH Zurich – Swiss Federal Institute of Techno...,ETH Zurich - Swiss Federal Institute of Techno...
9,"Nanyang Technological University, Singapore","Nanyang Technological University, Singapore (NTU)"


,name,title
0,Massachusetts Institute of Technology,Massachusetts Institute of Technology (MIT)
1,Stanford University,Stanford University
2,Harvard University,Harvard University
3,California Institute of Technology,California Institute of Technology (Caltech)
4,University of Cambridge,University of Cambridge
5,University of Oxford,University of Oxford
6,NaN,UCL (University College London)
7,Imperial College London,Imperial College London
8,University of Chicago,University of Chicago
9,ETH Zurich – Swiss Federal Institute of Techno...,ETH Zurich - Swiss Federal Institute of Techno...
